Prepared by Mike Mudimba NS2100042 (CSC 5741: Data Mining and Warehousing) Mini Project Practical Assignment

This Jupyter Notebook uses Selenium to extract comments on the Zambian Watchdog Facebook page. It automatically loops from a list of selected urls in the process extracting all:

1. User names
2. User comment
3. Number of reactions

It saves all extracted data to a scrapped_data.csv file

# Import necessary library files

In [19]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import os
import wget
import time

# Block Chrome alerts

In [20]:
#credits to pythonjar, stackoverflow
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 4}
chrome_options.add_experimental_option("prefs",prefs)

# Access facebook and login

To successfully scrap comments off Facebook, we will need to use Chromedriver to enable as read data passed through the Chrome web browser.

In [21]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome('C:/Users/mikem/chromedriver.exe', options=chrome_options)

#open the webpage
driver.get("http://www.facebook.com")

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("email_here")
password.clear()
password.send_keys("password_here")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

#We are logged in!

# Define the posts we will be scrapping data from

In [22]:
# We will extract data from 20 facebook posts on the facebook page

post = [
         "https://mbasic.facebook.com/story.php?story_fbid=4515405018510213&id=129987587052000&refid=17&_ft_=mf_story_key.4515405018510213%3Atop_level_post_id.4515405018510213%3Atl_objid.4515405018510213%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4515405018510213%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX_OmjwQiYoDv8P4%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A-7661448856687709388&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4513814035335978&id=129987587052000&refid=17&_ft_=mf_story_key.4513814035335978%3Atop_level_post_id.4513814035335978%3Atl_objid.4513814035335978%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4513814035335978%3Apage_id.129987587052000%3Aphoto_attachments_list.%5B4513813768669338%2C4513813882002660%5D%3Astory_location.4%3Astory_attachment_style.album%3Atds_flgs.3%3Aott.AX9w0iYy-ZJz0ULQ%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A-1381802933034242003&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4513384472045601&id=129987587052000&refid=17&_ft_=mf_story_key.4513384472045601%3Atop_level_post_id.4513384472045601%3Atl_objid.4513384472045601%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4513384472045601%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX-9yH2bnzmp7xWX%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A4764546387985999338&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4513270115390370&id=129987587052000&refid=17&_ft_=mf_story_key.4513270115390370%3Atop_level_post_id.4513270115390370%3Atl_objid.4513270115390370%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4513270115390370%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX9GndC9a72wb0Ua%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A9120059013044962067&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4514294881954560&id=129987587052000&refid=17&_ft_=mf_story_key.4514294881954560%3Atop_level_post_id.4514294881954560%3Atl_objid.4514294881954560%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4514294881954560%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.photo%3Atds_flgs.3%3Aott.AX8tXf_L9NisDZBC%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A5749355745092118280&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4516058265111555&id=129987587052000&refid=17&_ft_=mf_story_key.4516058265111555%3Atop_level_post_id.4516058265111555%3Atl_objid.4516058265111555%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4516058265111555%3Apage_id.129987587052000%3Aphoto_attachments_list.%5B4516057555111626%2C4516057581778290%2C4516057871778261%2C4516057868444928%5D%3Astory_location.4%3Astory_attachment_style.album%3Atds_flgs.3%3Aott.AX9uH-HVvHewHjMD%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A-224539940005134756&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4515997288450986&id=129987587052000&refid=17&_ft_=mf_story_key.4515997288450986%3Atop_level_post_id.4515997288450986%3Atl_objid.4515997288450986%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4515997288450986%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX8aDIK9vjg62gXt%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A-7318898135064293874&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4515693708481344&id=129987587052000&refid=17&_ft_=mf_story_key.4515693708481344%3Atop_level_post_id.4515693708481344%3Atl_objid.4515693708481344%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4515693708481344%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX_e_p-ONLVkmQ1S%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A4963303536925760714&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4519727581411290&id=129987587052000&refid=17&_ft_=mf_story_key.4519727581411290%3Atop_level_post_id.4519727581411290%3Atl_objid.4519727581411290%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4519727581411290%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX--rdKevoc8hORI%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A-5444579685592988049&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4519722528078462&id=129987587052000&refid=17&_ft_=mf_story_key.4519722528078462%3Atop_level_post_id.4519722528078462%3Atl_objid.4519722528078462%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4519722528078462%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX-zq5Dk80UrmzYY%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A7068669648020054871%3Apage_insights.%7B%22129987587052000%22%3A%7B%22page_id%22%3A129987587052000%2C%22page_id_type%22%3A%22page%22%2C%22actor_id%22%3A129987587052000%2C%22dm%22%3A%7B%22isShare%22%3A0%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntStatusCreationStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A266%2C%22publish_time%22%3A1628196996%2C%22story_name%22%3A%22EntStatusCreationStory%22%2C%22story_fbid%22%3A%5B4519722528078462%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A4%2C%22targets%22%3A%5B%7B%22actor_id%22%3A129987587052000%2C%22page_id%22%3A129987587052000%2C%22post_id%22%3A4519722528078462%2C%22role%22%3A1%2C%22share_id%22%3A0%7D%5D%7D%7D&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4518813088169406&id=129987587052000&refid=17&_ft_=mf_story_key.4518813088169406%3Atop_level_post_id.4518813088169406%3Atl_objid.4518813088169406%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4518813088169406%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX84t_-KuTrNN8OX%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A6075248826847843098%3Apage_insights.%7B%22129987587052000%22%3A%7B%22page_id%22%3A129987587052000%2C%22page_id_type%22%3A%22page%22%2C%22actor_id%22%3A129987587052000%2C%22dm%22%3A%7B%22isShare%22%3A0%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntStatusCreationStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A266%2C%22publish_time%22%3A1628172380%2C%22story_name%22%3A%22EntStatusCreationStory%22%2C%22story_fbid%22%3A%5B4518813088169406%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A4%2C%22targets%22%3A%5B%7B%22actor_id%22%3A129987587052000%2C%22page_id%22%3A129987587052000%2C%22post_id%22%3A4518813088169406%2C%22role%22%3A1%2C%22share_id%22%3A0%7D%5D%7D%7D&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=4518682384849143&id=129987587052000&refid=17&_ft_=mf_story_key.4518682384849143%3Atop_level_post_id.4518682384849143%3Atl_objid.4518682384849143%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.4518682384849143%3Apage_id.129987587052000%3Aphoto_attachments_list.%5B4518682034849178%2C4518682098182505%2C4518682171515831%2C4518682091515839%5D%3Astory_location.4%3Astory_attachment_style.album%3Atds_flgs.3%3Aott.AX9hOYo0xfT44dN_%3Athid.129987587052000%3A306061129499414%3A2%3A0%3A1630479599%3A9061917150044435860%3Apage_insights.%7B%22129987587052000%22%3A%7B%22page_id%22%3A129987587052000%2C%22page_id_type%22%3A%22page%22%2C%22actor_id%22%3A129987587052000%2C%22dm%22%3A%7B%22isShare%22%3A0%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntStatusCreationStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A266%2C%22publish_time%22%3A1628169224%2C%22story_name%22%3A%22EntStatusCreationStory%22%2C%22story_fbid%22%3A%5B4518682384849143%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A4%2C%22targets%22%3A%5B%7B%22actor_id%22%3A129987587052000%2C%22page_id%22%3A129987587052000%2C%22post_id%22%3A4518682384849143%2C%22role%22%3A1%2C%22share_id%22%3A0%7D%5D%7D%7D&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=3900316330019088&id=129987587052000&refid=17&_ft_=mf_story_key.3900316330019088%3Atop_level_post_id.3900316330019088%3Atl_objid.3900316330019088%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.3900316330019088%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX9UzA4X_W9RjGWL%3Athid.129987587052000%3A306061129499414%3A2%3A1577865600%3A1609487999%3A4055527268880093711&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=3899720956745292&id=129987587052000&refid=17&_ft_=mf_story_key.3899720956745292%3Atop_level_post_id.3899720956745292%3Atl_objid.3899720956745292%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.3899720956745292%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX-U7c9_wv6AO82P%3Athid.129987587052000%3A306061129499414%3A2%3A1577865600%3A1609487999%3A497324930107568536&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=3901054456611942&id=129987587052000&refid=17&_ft_=mf_story_key.3901054456611942%3Atop_level_post_id.3901054456611942%3Atl_objid.3901054456611942%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.3901054456611942%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.photo%3Atds_flgs.3%3Aott.AX-ndvKLc9qQ-7kx%3Athid.129987587052000%3A306061129499414%3A2%3A1577865600%3A1609487999%3A1701176032535452562&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=3902084193175635&id=129987587052000&refid=17&_ft_=mf_story_key.3902084193175635%3Atop_level_post_id.3902084193175635%3Atl_objid.3902084193175635%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.3902084193175635%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX-d3Q-rhnC4-Paj%3Athid.129987587052000%3A306061129499414%3A2%3A1577865600%3A1609487999%3A7208989457572617613&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=2918194631564601&id=129987587052000&refid=17&_ft_=mf_story_key.2918194631564601%3Atop_level_post_id.2918194631564601%3Atl_objid.2918194631564601%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.2918194631564601%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX_RqtAXyxFs9lpt%3Athid.129987587052000%3A306061129499414%3A2%3A1546329600%3A1577865599%3A4447231093150901077&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=2918121584905239&id=129987587052000&refid=17&_ft_=mf_story_key.2918121584905239%3Atop_level_post_id.2918121584905239%3Atl_objid.2918121584905239%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.2918121584905239%3Apage_id.129987587052000%3Astory_location.4%3Atds_flgs.3%3Aott.AX9Ywtms5_ILs6ur%3Athid.129987587052000%3A306061129499414%3A2%3A1546329600%3A1577865599%3A3386951765836696431&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=2918071071576957&id=129987587052000&refid=17&_ft_=mf_story_key.2918071071576957%3Atop_level_post_id.2918071071576957%3Atl_objid.2918071071576957%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.2918071071576957%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.video_inline%3Atds_flgs.3%3Aott.AX_M1jotvhScBpI8%3Athid.129987587052000%3A306061129499414%3A2%3A1546329600%3A1577865599%3A8785660007542673756&__tn__=%2AW-R/",
         "https://mbasic.facebook.com/story.php?story_fbid=2917659048284826&id=129987587052000&refid=17&_ft_=mf_story_key.2917659048284826%3Atop_level_post_id.2917659048284826%3Atl_objid.2917659048284826%3Acontent_owner_id_new.129987587052000%3Athrowback_story_fbid.2917659048284826%3Apage_id.129987587052000%3Astory_location.4%3Astory_attachment_style.photo%3Atds_flgs.3%3Aott.AX_rlpgJqMVwAAew%3Athid.129987587052000%3A306061129499414%3A2%3A1546329600%3A1577865599%3A4209279681851065478&__tn__=%2AW-R/"
        ]

# Define element attributes for wedriver to use for grabbing data

In order to grad data, we need to know the elements where this data is defined. The challenge however is, facebook constantly changes and refreshes the naming of these attributes. Thus, we need to define a list as per page that we shall use to pull data.

Everytime you run this project, this is the last sction that you need to configure according to the attributes showing in the Google Chrome developer tools that you wish to pool

In [23]:
# Attributes include name, comment, reaction
attribute = [
                ["ds bf","dt","dz bp"],
                ["dz bi","ea","ei bx"],
                ["dy bf","dz","ef bv"],
                ["dy bf","dz","ef bv"],
                ["dz bf","ea","eg bw"],
                ["dz bf","ea","eg bw"],
                ["ea bf","eb","eh bx"],
                ["dy bf","dz","ef bv"],
                ["ds bf","dt","dz bp"],
                ["dy bf","dz","ef bv"],
                ["dy bi","dz","ef bw"],
                ["ee cy","ef","el cb"],
                ["dy bf","dz","ef bv"],
                ["ds bi","dt","dz bq"],
                ["dz ct","ea","eg bw"],
                ["ds bi","dt","dz bq"],
                ["ds bf","dt","dz bp"],
                ["ds bf","dt","dz bp"],
                ["dy bi","dz","ef bw"],
                ["dz bf","ea","eg bw"]
            ]

# Initialize return variable where scrapped data will be stored

In [24]:
# Return variable will be a list that can be converted to csv or Excel later on
return_variable = [
    ["Name","Comment","Reactions"]
]

# Loop through posts and attributes to fetch data

In [25]:
for post_url, attribute_list in zip(post, attribute):
    
    #navigate to post
    driver.get(post_url)
    
    #get attributes
    name_attr = attribute_list[0]
    comment_attr = attribute_list[1]
    react_attr = attribute_list[2]
    
    #grab data on page
    names = driver.find_elements_by_xpath("//a[@class='" + name_attr + "']")
    comments = driver.find_elements_by_xpath("//div[@class='" + comment_attr + "']")
    reactions = driver.find_elements_by_xpath("//a[@class='" + react_attr + "']")
    
    for name, comment, reaction in zip(names, comments, reactions):
        row = [name.text, comment.text, reaction.text]
        return_variable.append(row)

# Convert results into a csv file

In [26]:
#We will use the panda lib to convert results into a csv file (scrapped_data.csv)
import pandas as pd
df = pd.DataFrame(return_variable)
df.to_csv('scrapped_data.csv', index=False, header=False)

# Print the results

In [27]:
print(return_variable)

[['Pumulo Mayungo', 'This is the reason why I trust and respect soldiers. They will never let us down as citizens not even a single day unlike the police. Theirs is to protect citizens with impartiality regardless of who is involved in acts of violence. 👏👏👏', '159'], ['Collins Kafunga', 'Well done zambia army your loyalty is to the people and the constitution of the republic not a certain grouping of corrupt individuals like other security wings does', '75'], ['John Marks', 'These PF cadres actually complained to their SG Davies Mwila ati nomba won\'t the soldiers start harrassing us because of using military titles. Then Davies assured them that they won\'t be touched. I still think Lungu is trying to use the army to rig but it might backfire,if the military doesn\'t tolerate nonsense. Nomba the army commander Sikazwe is compromised they should just do a mutiny against him if he asks them to do "funny rigging things".', '16'], ['Barbra M Chembo', 'The moment I heard soldiers are deplo

# This marks the end of data collection